# Description

This notebook calculates a one-sided poisson confidence interval for event occurrence rate given the ```test_duration```, the number of events observed ```n_events``` during that test, and a desired ```confidence_level``` for the event occurrence rate.

When faced with the result of a continuous test (for example the number of false positives observed during 10 hours), this tells you what you should expect the real events occurrence rate (if you ran the test infinitely long) to be. You can tune how conservative or optimistic this estimate for the event occurrence rate should be by setting the ```confidence level```. If you set the confidence level to ```0.95```, the real event occurrence rate will lie within the interval calculated at least in ```95%``` of these evaluations. That means there still is at most a ```5%``` chance that the real event occurrence rate will be worse than the estimated interval suggests.

__Note__
1. All probabilities are given as numbers between 0 and 1, where 0 represents "never" and 1 represents 100% certainty.
2. The confidence intervals are conservative, i.e., the coverage of this hypothesis test is ```confidence_level``` or higher. That means if you use this notebook with ```confidence_level = 0.95``` for multiple trials, then you should expect the true event occurrence rate to lie outside the reported confidence interval in 1/20 times on average in the worst-case.

# Parameters

In [1]:
test_duration = 10 #measure of how long the test was run [km, hours, ...]
n_events = 0 #number of events observed during test_duration
confidence_level = 0.9

round_to_digits = 4

assert isinstance(test_duration, (int, float)) and test_duration > 0
assert isinstance(n_events, int) and n_events >= 0
assert isinstance(confidence_level, float) and 0 <= confidence_level <= 1
assert isinstance(round_to_digits, int) and round_to_digits >= 1

# Evaluation

In [2]:
import matplotlib.pyplot as plt
import mpmath as mp
import numpy as np
import scipy.optimize as opt

mp.mp.dps = 50 #use 50 decimal digits precision in mpmath calculations

def poisson_distr(n_events, r_event, test_duration):
    e = mp.mpf(float(r_event)) * mp.mpf(float(test_duration))
    return mp.power(mp.mpf(e), n_events) / mp.fac(n_events) * mp.exp(-e)

poisson_distr_vectorized = np.vectorize(poisson_distr) #create version of poisson distribution that can be evaluated for whole lists of inputs

def poisson_confidence(n_events, r_hypothesis, test_duration):
    return 1-poisson_distr_vectorized(np.arange(0, n_events+1), r_hypothesis, test_duration).sum()

r_min = opt.root_scalar(lambda r: float(poisson_confidence(n_events, r, test_duration)-confidence_level), x0=(n_events+1)/test_duration, method='newton')
r_min = mp.nstr(mp.mpf(r_min.root), round_to_digits)

print('The event occurrence rate lies within [0, {}] with a confidence of {}'.format(r_min, confidence_level))

The event occurrence rate lies within [0, 0.2303] with a confidence of 0.9
